# Extract Contours
This notebook was a rehearsal to extract contours from an image and paste them again.
With the idea of doing some processing in between to the crop, like Unet-segmentation.


#### Part 1 - Extract the contours

In [ ]:
import tifffile as tiff
from natsort import natsorted
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cv2
from imutils.src import imfunctions #contours_length as imutils.contours_length
from natsort import natsorted
import shutil
import pandas as pd
import glob

In [ ]:
img_path='2020-07-01_17-32-12_control_worm4-channel-0-bigtiff_img017777.tif'
#img_path='image_0_151189.png'
img=cv2.imread(img_path,0)
print(img.shape)

In [ ]:
#image basename
img_basename=os.path.splitext(os.path.basename(img_path))[0]
print(img_basename)

In [ ]:
plt.imshow(img)

In [ ]:
#median Blur
img=cv2.medianBlur(img,5)

#apply threshold
ret, new_img = cv2.threshold(img,20,255,cv2.THRESH_BINARY)
#get some contours
worm_contour_img=imfunctions.draw_some_contours(new_img,contour_size=8000,tolerance=0.25, inner_contour_area_to_fill=300)

In [ ]:
plt.imshow(worm_contour_img)

In [ ]:
#find external contours on the binary image
_,cnts,hierarchy=cv2.findContours(worm_contour_img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#draw them
print(len(cnts))

In [ ]:
x_list,y_list,w_list,h_list=[],[],[],[]
for cnt_idx,cnt in enumerate(cnts):
    #create an empty image equal to the source
    cnt_img=np.zeros(shape=img.shape)
    #draw the contours on it
    cv2.drawContours(cnt_img, cnts, np.array(cnt_idx),0,-1)
    #get the boundingrectangle of the contour
    x,y,w,h = cv2.boundingRect(cnt)
    cnt_img=worm_contour_img[y:y+h,x:x+w]

    # Create dictionary my_dict with three key:value pairs: my_dict
    my_dict={'x':[x],
            'y':[y],
            'width':[w],
             'height': [h]
    }

    # Build a DataFrame cars from my_dict: cars
    df=pd.DataFrame(my_dict)
    df.to_csv(img_basename+'_cnt_'+str(cnt_idx)+'.csv')
    
    tiff.imwrite(img_basename+'_cnt_'+str(cnt_idx)+'.tiff',cnt_img)


In [ ]:
plt.figure(dpi=100)
plt.imshow(worm_contour_img)
plt.axis('off')
img.shape

#### Part 2 - Paste the contours onto an image


In [ ]:
#
#PASTING THE PROCESSED CROP BACK TO THE IMAGE

img_shape=(610,608) #not needed probably. Maybe only to double check
crops_path='/groups/zimmer/Ulises/code/imutils/imutils/dev/crops/'

#get all png files in there
list_of_img_crops=natsorted(glob.glob((os.path.join(crops_path, '*.tiff'))))
list_of_csv_crops=natsorted(glob.glob((os.path.join(crops_path, '*.csv'))))
for idx,file in enumerate(list_of_img_crops):
    print(file)
    crop=tiff.imread(os.path.join(crops_path,file))
    df=pd.read_csv(os.path.join(crops_path,list_of_csv_crops[idx]),index_col=0)
    coords_list=df.values.tolist()[0]
    mother_img=np.zeros(shape=img_shape)
    pasted_img=paste_img(mother_img, crop, coords_list)
    tiff.imwrite(file, pasted_img)

In [ ]:
plt.imshow(pasted_img)

In [ ]:
def paste_img(img, crop, coords_list):
    """
    Paste a crop into an image
    Parameters:
    -----------
    img, np.array
    crop, np.array
    coords_tuple, tuple with (x,y,w,h)
    As provided by cv2.BoundingRect for example. Maybe x and y are good enough
    Returns:
    -----------
    img, np.array
    Pasted image
    """
    x,y,w,h=coords_list
    #try:
    img[y:y+h,x:x+w]=crop
    #except: print('Problem with the dimensions of the crop')
    return img